In [1]:
import numpy as np

In [2]:
class BasicNeuron:
    """
    A basic artificial neuron implementation that mimics biological neurons.

    The neuron receives inputs, applies weights, adds bias, and produces an output
    through an activation function - just like neurons in our brain!
    """

    def __init__(self, num_inputs, activation_function='sigmoid'):
        """
        Initialize the neuron with random weights and bias.

        Args:
            num_inputs: Number of input connections to this neuron
            activation_function: Type of activation function ('sigmoid', 'relu', 'tanh', 'linear')
        """
        # Initialize weights randomly between -1 and 1
        # Each input gets its own weight - this determines how important each input is
        self.weights = np.random.uniform(-1, 1, num_inputs)

        # Initialize bias - this shifts the activation function left or right
        # Bias helps the neuron fire even when inputs are small
        self.bias = np.random.uniform(-1, 1)

        # Store the activation function type
        if activation_function not in ['sigmoid', 'relu', 'tanh', 'linear']:
            raise ValueError(f"Unsupported activation function: {activation_function}")
        self.activation_function = activation_function

        # Store the number of inputs for validation
        self.num_inputs = num_inputs

        # print(f"Neuron created with {num_inputs} inputs")
        # print(f"Initial weights: {self.weights}")
        # print(f"Initial bias: {self.bias}")
        # print(f"Activation function: {activation_function}")

    def sigmoid(self, x):
        """
        Sigmoid activation function: f(x) = 1 / (1 + e^(-x))

        - Outputs values between 0 and 1
        - Smooth, differentiable curve
        - Good for binary classification problems
        """
        # Clip x to prevent overflow in exponential
        x = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """Derivative of the sigmoid function."""
        # Derivative of sigmoid(x) is sigmoid(x) * (1 - sigmoid(x))
        # We can use the output of the sigmoid function itself for this
        s = self.sigmoid(x)
        return s * (1 - s)

    def relu(self, x):
        """ReLU activation function: f(x) = max(0, x)"""
        return np.maximum(0, x)

    def relu_derivative(self, x):
        """Derivative of the ReLU function."""
        return np.where(x > 0, 1, 0)

    def tanh(self, x):
        """Tanh activation function: f(x) = tanh(x)"""
        return np.tanh(x)

    def tanh_derivative(self, x):
        """Derivative of the Tanh function."""
        # Derivative of tanh(x) is 1 - tanh(x)^2
        return 1 - np.tanh(x)**2

    def linear(self, x):
        """Linear activation function: f(x) = x"""
        return x

    def linear_derivative(self, x):
        """Derivative of the Linear function."""
        return 1

    def get_activation_derivative(self, z):
        """Returns the derivative of the chosen activation function for a given net input z."""
        if self.activation_function == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif self.activation_function == 'relu':
            return self.relu_derivative(z)
        elif self.activation_function == 'tanh':
            return self.tanh_derivative(z)
        elif self.activation_function == 'linear':
            return self.linear_derivative(z)
        else:
            raise ValueError(f"Derivative not implemented for {self.activation_function}")


    def forward(self, inputs):

        # Convert inputs to numpy array for easier computation
        inputs = np.array(inputs)

        # Validate input size
        if len(inputs) != self.num_inputs:
            raise ValueError(f"Expected {self.num_inputs} inputs, got {len(inputs)}")

        # Step 1 & 2: Weighted sum (dot product of inputs and weights)
        # This is like: w1*x1 + w2*x2 + w3*x3 + ... + wn*xn
        weighted_sum = np.dot(inputs, self.weights)

        # Step 3: Add bias
        # Bias allows the neuron to fire even when inputs are zero
        z = weighted_sum + self.bias

        # Step 4: Apply activation function
        if self.activation_function == 'sigmoid':
            output = self.sigmoid(z)
        elif self.activation_function == 'relu':
            output = self.relu(z)
        elif self.activation_function == 'tanh':
            output = self.tanh(z)
        elif self.activation_function == 'linear':
            output = self.linear(z)
        else:
            raise ValueError(f"Unknown activation function: {self.activation_function}")

        # Store intermediate values for educational purposes
        self.last_inputs = inputs
        self.last_weighted_sum = weighted_sum
        self.last_z = z # Store z (net input before activation) for derivative calculation
        self.last_output = output

        return output

    def update_weights(self, new_weights, new_bias=None):
        if len(new_weights) != self.num_inputs:
            raise ValueError(f"Expected {self.num_inputs} weights, got {len(new_weights)}")

        self.weights = np.array(new_weights)

        if new_bias is not None:
            self.bias = new_bias

        # print(f"Weights updated to: {self.weights}")
        # print(f"Bias updated to: {self.bias}")

    def get_details(self, verbose=True):
        if hasattr(self, 'last_inputs'):
            if verbose:
                print("\n--- Neuron Computation Details ---")
                print(f"Inputs: {self.last_inputs}")
                print(f"Weights: {self.weights}")
                print(f"Weighted sum: {self.last_weighted_sum:.4f}")
                print(f"Bias: {self.bias:.4f}")
                print(f"z (weighted sum + bias): {self.last_z:.4f}")
                print(f"Final output: {self.last_output:.4f}")
        else:
            if verbose:
                print("No computation has been performed yet!")


In [20]:
#correlation

def correlation_learning(inputs, outputs, learning_rate=1.0):
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')
    neuron.update_weights(np.zeros(num_inputs), 0.0)

    num_patterns = inputs.shape[0]

    print(f"\n--- Correlation Learning ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for i in range(num_patterns):
        x = inputs[i]
        y_desired = outputs[i]

        delta_weights = learning_rate * np.outer(x, [y_desired]).flatten()

        new_weights = neuron.weights + delta_weights
        new_bias = neuron.bias 

        neuron.update_weights(new_weights, new_bias)

        print(f"Pattern {i+1}: Input={x}, Desired Output={y_desired}")
        print(f"  Delta Weights: {delta_weights}")
        print(f"  Updated Weights: {neuron.weights}, Updated Bias: {neuron.bias}")

    print(f"Final Weights (Correlation): {neuron.weights}, Final Bias: {neuron.bias}")
    return neuron.weights

In [21]:
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
outputs_and = np.array([0,0,0,1])

print("\n=== Correlation Learning on AND Gate ===")
w = correlation_learning(inputs, outputs_and, learning_rate=1.0)

print("\n--- Testing Learned Weights on AND Gate ---")
for activation in ["linear", "sigmoid", "tanh", "relu"]:
    neuron = BasicNeuron(num_inputs=2, activation_function=activation)
    neuron.update_weights(w, 0.0)
    preds = [neuron.forward(x) for x in inputs]
    print(f"\nActivation: {activation}")
    for x, p in zip(inputs, preds):
        print(f"Input {x} -> Output {p:.4f}")

outputs_or = np.array([0,1,1,1])
print("\n=== Correlation Learning on OR Gate ===")
w = correlation_learning(inputs, outputs_or, learning_rate=1.0)

print("\n--- Testing Learned Weights on OR Gate ---")
for activation in ["linear", "sigmoid", "tanh", "relu"]:
    neuron = BasicNeuron(num_inputs=2, activation_function=activation)
    neuron.update_weights(w, 0.0)
    preds = [neuron.forward(x) for x in inputs]
    print(f"\nActivation: {activation}")
    for x, p in zip(inputs, preds):
        print(f"Input {x} -> Output {p:.4f}")



=== Correlation Learning on AND Gate ===

--- Correlation Learning ---
Initial Weights: [0. 0.], Initial Bias: 0.0
Pattern 1: Input=[0 0], Desired Output=0
  Delta Weights: [0. 0.]
  Updated Weights: [0. 0.], Updated Bias: 0.0
Pattern 2: Input=[0 1], Desired Output=0
  Delta Weights: [0. 0.]
  Updated Weights: [0. 0.], Updated Bias: 0.0
Pattern 3: Input=[1 0], Desired Output=0
  Delta Weights: [0. 0.]
  Updated Weights: [0. 0.], Updated Bias: 0.0
Pattern 4: Input=[1 1], Desired Output=1
  Delta Weights: [1. 1.]
  Updated Weights: [1. 1.], Updated Bias: 0.0
Final Weights (Correlation): [1. 1.], Final Bias: 0.0

--- Testing Learned Weights on AND Gate ---

Activation: linear
Input [0 0] -> Output 0.0000
Input [0 1] -> Output 1.0000
Input [1 0] -> Output 1.0000
Input [1 1] -> Output 2.0000

Activation: sigmoid
Input [0 0] -> Output 0.5000
Input [0 1] -> Output 0.7311
Input [1 0] -> Output 0.7311
Input [1 1] -> Output 0.8808

Activation: tanh
Input [0 0] -> Output 0.0000
Input [0 1] -> Ou

In [ ]:


def outstar_learning(inputs, outputs, learning_rate=0.1, max_epochs=100, tolerance=1e-6):
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')
    
    neuron.update_weights(np.zeros(num_inputs), 0.0)
    
    print("\n--- Outstar Learning with Convergence ---")
    print(f"Initial Weights: {neuron.weights}")
    
    for epoch in range(max_epochs):
        total_change = 0.0
        
        for i in range(inputs.shape[0]):
            y_desired = outputs[i]
            
            delta_w = learning_rate * (y_desired - neuron.weights)
            new_weights = neuron.weights + delta_w
            
            total_change += np.linalg.norm(delta_w)
            
            neuron.update_weights(new_weights, neuron.bias) 
        if total_change < tolerance:
            print(f"Converged after {epoch+1} epochs.")
            break
    
    print(f"Final Weights: {neuron.weights}")
    return neuron.weights


In [22]:

inputs = np.array([[1, 0],
                    [0, 1],
                    [1, 1]])  
target = np.array([0.8, 0.2])  
outputs_single = np.tile(target, (inputs.shape[0], 1))  
print("\n=== Outstar Learning: Single Target Vector (Tiled) ===")
w1 = outstar_learning(inputs, outputs_single, learning_rate=0.2, max_epochs=50)
print("Learned weights (should approach target):", w1)

outputs_multi = np.array([
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
])  
print("\n=== Outstar Learning: Multiple Target Vectors ===")
w2 = outstar_learning(inputs, outputs_multi, learning_rate=0.1, max_epochs=100)
print("Learned weights (should approach average of rows):", w2)

for activation in ["linear", "sigmoid", "tanh", "relu"]:
    neuron = BasicNeuron(num_inputs=2, activation_function=activation)
    neuron.update_weights(w2, 0.0)
    preds = [neuron.forward(x) for x in inputs]
    print(f"\nActivation: {activation}")
    for x, p in zip(inputs, preds):
        print(f"Input {x} -> Output {p}")



=== Outstar Learning: Single Target Vector (Tiled) ===

--- Outstar Learning with Convergence ---
Initial Weights: [0. 0.]
Converged after 21 epochs.
Final Weights: [0.79999937 0.19999984]
Learned weights (should approach target): [0.79999937 0.19999984]

=== Outstar Learning: Multiple Target Vectors ===

--- Outstar Learning with Convergence ---
Initial Weights: [0. 0.]
Final Weights: [0.66789668 0.70110701]
Learned weights (should approach average of rows): [0.66789668 0.70110701]

Activation: linear
Input [1 0] -> Output 0.6678966789667772
Input [0 1] -> Output 0.7011070110700977
Input [1 1] -> Output 1.369003690036875

Activation: sigmoid
Input [1 0] -> Output 0.6610320305705558
Input [0 1] -> Output 0.668433165059571
Input [1 1] -> Output 0.7972191370124587

Activation: tanh
Input [1 0] -> Output 0.5835946189964413
Input [0 1] -> Output 0.6050699707861333
Input [1 1] -> Output 0.8784649354211677

Activation: relu
Input [1 0] -> Output 0.6678966789667772
Input [0 1] -> Output 0.70

In [23]:
def instar_learning(inputs, num_neurons=2, learning_rate=0.5, epochs=10): 
    num_inputs = inputs.shape[1]
    neurons = [BasicNeuron(num_inputs, activation_function='linear') for _ in range(num_neurons)]

    print(f"\n--- Instar (Winner-Take-All) Learning Law ---")
    for i, n in enumerate(neurons):
        print(f"Initial Weights for Neuron {i+1}: {n.weights}")

    for epoch in range(epochs):
        print(f"\n--- Epoch {epoch+1} ---")
        for x in inputs:
            net_inputs = [np.dot(n.weights, x) for n in neurons]
            winner_index = np.argmax(net_inputs)
            winner_neuron = neurons[winner_index]

            delta_weights = learning_rate * (x - winner_neuron.weights)
            new_weights = winner_neuron.weights + delta_weights
            winner_neuron.update_weights(new_weights)
            
            print(f"Input: {x}, Winner: Neuron {winner_index+1}, Updated Weights: {winner_neuron.weights}")

    print("\nFinal Weights (Instar):")
    for i, n in enumerate(neurons):
        print(f"Neuron {i+1}: {n.weights}")
    return [n.weights for n in neurons]


In [24]:
competitive_inputs = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [0, 0]
])
instar_learning(competitive_inputs)


--- Instar (Winner-Take-All) Learning Law ---
Initial Weights for Neuron 1: [0.05919776 0.38242316]
Initial Weights for Neuron 2: [-0.36608734 -0.18342443]

--- Epoch 1 ---
Input: [1 0], Winner: Neuron 1, Updated Weights: [0.52959888 0.19121158]
Input: [0 1], Winner: Neuron 1, Updated Weights: [0.26479944 0.59560579]
Input: [1 1], Winner: Neuron 1, Updated Weights: [0.63239972 0.79780289]
Input: [0 0], Winner: Neuron 1, Updated Weights: [0.31619986 0.39890145]

--- Epoch 2 ---
Input: [1 0], Winner: Neuron 1, Updated Weights: [0.65809993 0.19945072]
Input: [0 1], Winner: Neuron 1, Updated Weights: [0.32904997 0.59972536]
Input: [1 1], Winner: Neuron 1, Updated Weights: [0.66452498 0.79986268]
Input: [0 0], Winner: Neuron 1, Updated Weights: [0.33226249 0.39993134]

--- Epoch 3 ---
Input: [1 0], Winner: Neuron 1, Updated Weights: [0.66613125 0.19996567]
Input: [0 1], Winner: Neuron 1, Updated Weights: [0.33306562 0.59998284]
Input: [1 1], Winner: Neuron 1, Updated Weights: [0.66653281 0

[array([0.33333333, 0.4       ]), array([-0.36608734, -0.18342443])]

In [25]:

def reinforcement_learning(inputs, rewards, learning_rate=1.0):
   
    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='linear')
    neuron.update_weights(np.zeros(num_inputs), 0.0)

    num_patterns = inputs.shape[0]

    print(f"\n--- Reinforcement Learning ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for i in range(num_patterns):
        x = inputs[i]
        r = rewards[i]

        y = neuron.forward(x)

        delta_weights = learning_rate * r * x

        new_weights = neuron.weights + delta_weights
        new_bias = neuron.bias  

        neuron.update_weights(new_weights, new_bias)

        print(f"Pattern {i+1}: Input={x}, Reward={r}, Output={y}")
        print(f"  Delta Weights: {delta_weights}")
        print(f"  Updated Weights: {neuron.weights}, Updated Bias: {neuron.bias}")

    print(f"Final Weights: {neuron.weights}, Final Bias: {neuron.bias}")

    test_output = neuron.forward(inputs[-1])
    print(f"Test output for last pattern: {test_output}")

    return neuron.weights


In [27]:
inputs = np.array([[1, 0],
                       [0, 1],
                       [1, 1]])  
target = np.array([0.8, 0.2])  
outputs_single = np.tile(target, (inputs.shape[0], 1))  

print("\n=== Reinforcement Learning: Single Target Vector (Tiled) ===")
w1 = reinforcement_learning(inputs, outputs_single, learning_rate=0.2)
print("Learned weights (should approach target):", w1)

outputs_multi = np.array([
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
])  

print("\n=== Reinforcement Learning: Multiple Target Vectors ===")
w2 = reinforcement_learning(inputs, outputs_multi, learning_rate=0.1)
print("Learned weights (should approach average of rows):", w2)

for activation in ["linear", "sigmoid", "tanh", "relu"]:
    neuron = BasicNeuron(num_inputs=2, activation_function=activation)
    neuron.update_weights(w2, 0.0)
    preds = [neuron.forward(x) for x in inputs]
    print(f"\nActivation: {activation}")
    for x, p in zip(inputs, preds):
        print(f"Input {x} -> Output {p}")


=== Reinforcement Learning: Single Target Vector (Tiled) ===

--- Reinforcement Learning ---
Initial Weights: [0. 0.], Initial Bias: 0.0
Pattern 1: Input=[1 0], Reward=[0.8 0.2], Output=0.0
  Delta Weights: [0.16 0.  ]
  Updated Weights: [0.16 0.  ], Updated Bias: 0.0
Pattern 2: Input=[0 1], Reward=[0.8 0.2], Output=0.0
  Delta Weights: [0.   0.04]
  Updated Weights: [0.16 0.04], Updated Bias: 0.0
Pattern 3: Input=[1 1], Reward=[0.8 0.2], Output=0.20000000000000004
  Delta Weights: [0.16 0.04]
  Updated Weights: [0.32 0.08], Updated Bias: 0.0
Final Weights: [0.32 0.08], Final Bias: 0.0
Test output for last pattern: 0.4000000000000001
Learned weights (should approach target): [0.32 0.08]

=== Reinforcement Learning: Multiple Target Vectors ===

--- Reinforcement Learning ---
Initial Weights: [0. 0.], Initial Bias: 0.0
Pattern 1: Input=[1 0], Reward=[1. 0.], Output=0.0
  Delta Weights: [0.1 0. ]
  Updated Weights: [0.1 0. ], Updated Bias: 0.0
Pattern 2: Input=[0 1], Reward=[0. 1.], Outp

In [28]:
def boltzmann_learning(inputs, outputs, learning_rate=0.1, epochs=10):

    num_inputs = inputs.shape[1]
    num_outputs = outputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function='sigmoid')

    init_w = np.random.uniform(-0.1, 0.1, (num_inputs, num_outputs))
    neuron.update_weights(init_w, 0.0)

    print("\n--- Boltzmann Learning ---")
    print(f"Initial Weights:\n{neuron.weights}\nInitial Bias: {neuron.bias}")

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}:")

        for i in range(inputs.shape[0]):
            x = inputs[i].reshape(-1, 1)        
            y_clamped = outputs[i].reshape(1,-1) 

            clamped_corr = np.dot(x, y_clamped) 
            y_free = neuron.forward(inputs[i])   
            y_free = np.array(y_free).reshape(1,-1)
            free_corr = np.dot(x, y_free)        
            delta_weights = learning_rate * (clamped_corr - free_corr)

            new_weights = neuron.weights + delta_weights
            neuron.update_weights(new_weights, neuron.bias)

            print(f"  Pattern {i+1}: Input={inputs[i]}, Clamped={y_clamped.flatten()}, Free={y_free.flatten()}")
            print(f"    ΔW:\n{delta_weights}")
            print(f"    Updated W:\n{neuron.weights}")

    print(f"\nFinal Weights (Boltzmann):\n{neuron.weights}\nFinal Bias: {neuron.bias}")
    return neuron.weights

In [29]:
inputs = np.array([[1, 0],
                       [0, 1],
                       [1, 1]])  
target = np.array([0.8, 0.2])  
outputs_single = np.tile(target, (inputs.shape[0], 1))  

print("\n=== Boltzmann Learning: Single Target Vector (Tiled) ===")
w1 = boltzmann_learning(inputs, outputs_single, learning_rate=0.2, epochs=50)
print("Learned weights (should approach target):", w1)

outputs_multi = np.array([
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
]) 

print("\n=== Boltzmann Learning: Multiple Target Vectors ===")
w2 = boltzmann_learning(inputs, outputs_multi, learning_rate=0.1, epochs=100)
print("Learned weights (should approach average of rows):", w2)

for activation in ["linear", "sigmoid", "tanh", "relu"]:
    neuron = BasicNeuron(num_inputs=2, activation_function=activation)
    neuron.update_weights(w2, 0.0)
    preds = [neuron.forward(x) for x in inputs]
    print(f"\nActivation: {activation}")
    for x, p in zip(inputs, preds):
        print(f"Input {x} -> Output {p}")


=== Boltzmann Learning: Single Target Vector (Tiled) ===

--- Boltzmann Learning ---
Initial Weights:
[[-0.0303209  -0.02930894]
 [ 0.03395626  0.04817207]]
Initial Bias: 0.0

Epoch 1:
  Pattern 1: Input=[1 0], Clamped=[0.8 0.2], Free=[0.49242035 0.49267329]
    ΔW:
[[ 0.06151593 -0.05853466]
 [ 0.          0.        ]]
    Updated W:
[[ 0.03119502 -0.08784359]
 [ 0.03395626  0.04817207]]
  Pattern 2: Input=[0 1], Clamped=[0.8 0.2], Free=[0.50848825 0.51204069]
    ΔW:
[[ 0.          0.        ]
 [ 0.05830235 -0.06240814]]
    Updated W:
[[ 0.03119502 -0.08784359]
 [ 0.09225861 -0.01423607]]
  Pattern 3: Input=[1 1], Clamped=[0.8 0.2], Free=[0.53082427 0.47450222]
    ΔW:
[[ 0.05383515 -0.05490044]
 [ 0.05383515 -0.05490044]]
    Updated W:
[[ 0.08503017 -0.14274404]
 [ 0.14609376 -0.06913651]]

Epoch 2:
  Pattern 1: Input=[1 0], Clamped=[0.8 0.2], Free=[0.52124474 0.46437446]
    ΔW:
[[ 0.05575105 -0.05287489]
 [ 0.          0.        ]]
    Updated W:
[[ 0.14078122 -0.19561893]
 [ 0

In [42]:
def delta_rule_learning(inputs, outputs, learning_rate=0.1, epochs=100, activation_func='linear'):

    num_inputs = inputs.shape[1]
    neuron = BasicNeuron(num_inputs, activation_function=activation_func)

    num_patterns = inputs.shape[0]

    print(f"\n--- Delta Rule Learning ({activation_func.capitalize()} Activation) ---")
    print(f"Initial Weights: {neuron.weights}, Initial Bias: {neuron.bias}")

    for epoch in range(epochs):
        total_error = 0
        for i in range(num_patterns):
            x = inputs[i]
            y_desired = outputs[i]

            y_actual = neuron.forward(x)

            error = y_desired - y_actual
            total_error += 0.5 * (error ** 2) 

            derivative_activation = neuron.get_activation_derivative(neuron.last_z)

            new_weights = neuron.weights + learning_rate * error * x * derivative_activation
            new_bias = neuron.bias + learning_rate * error * derivative_activation
            neuron.update_weights(new_weights, new_bias)

        if total_error < 0.001:
            print(f"Converged at Epoch {epoch+1}.")
            break
    else:
        print(f"Did not converge within {epochs} epochs.")

    print(f"Final Weights (Delta Rule): {neuron.weights}, Final Bias: {neuron.bias}")

    print(f"\n--- Delta Rule Test ({activation_func.capitalize()} Activation) ---")
    for i in range(num_patterns):
        x = inputs[i]
        y_predicted = neuron.forward(x)
        print(f"Input: {x}, Desired: {outputs[i]:.4f}, Predicted: {y_predicted:.4f}")


In [43]:
inputs_and = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

outputs_and = np.array([0, 0, 0, 1])  
print("\n=== Delta Rule Learning: AND Function (Linear Activation) ===")
delta_rule_learning(inputs_and, outputs_and, learning_rate=0.1, epochs=100, activation_func='linear')

print("\n=== Delta Rule Learning: AND Function (Sigmoid Activation) ===")
delta_rule_learning(inputs_and, outputs_and, learning_rate=0.5, epochs=200, activation_func='sigmoid')

inputs_or = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

outputs_or = np.array([0, 1, 1, 1]) 
print("\n=== Delta Rule Learning: OR Function (Linear Activation) ===")
delta_rule_learning(inputs_or, outputs_or, learning_rate=0.1, epochs=100, activation_func='linear')

print("\n=== Delta Rule Learning: OR Function (Sigmoid Activation) ===")
delta_rule_learning(inputs_or, outputs_or, learning_rate=0.5, epochs=200, activation_func='sigmoid')


inputs_map = np.array([
    [0],
    [1],
    [2],
    [3]
])

outputs_map = np.array([0.0, 0.5, 1.0, 1.5])
print("\n=== Delta Rule Learning: Continuous Mapping (Linear Activation) ===")
delta_rule_learning(inputs_map, outputs_map, learning_rate=0.05, epochs=300, activation_func='linear')


=== Delta Rule Learning: AND Function (Linear Activation) ===

--- Delta Rule Learning (Linear Activation) ---
Initial Weights: [-0.57795463 -0.54665802], Initial Bias: -0.018763079381051373
Did not converge within 100 epochs.
Final Weights (Delta Rule): [0.55481171 0.5269935 ], Final Bias: -0.27674641068131295

--- Delta Rule Test (Linear Activation) ---
Input: [0 0], Desired: 0.0000, Predicted: -0.2767
Input: [0 1], Desired: 0.0000, Predicted: 0.2502
Input: [1 0], Desired: 0.0000, Predicted: 0.2781
Input: [1 1], Desired: 1.0000, Predicted: 0.8051

=== Delta Rule Learning: AND Function (Sigmoid Activation) ===

--- Delta Rule Learning (Sigmoid Activation) ---
Initial Weights: [-0.87249238 -0.27542655], Initial Bias: -0.17848098951579927
Did not converge within 200 epochs.
Final Weights (Delta Rule): [2.61599785 2.59779902], Final Bias: -4.001329129615857

--- Delta Rule Test (Sigmoid Activation) ---
Input: [0 0], Desired: 0.0000, Predicted: 0.0180
Input: [0 1], Desired: 0.0000, Predi